## fastpredict

In [ ]:
!pip install transformers torch pandas scikit-learn

In [ ]:
import pandas as pd
import torch
from transformers import pipeline
import time
from sklearn.metrics import f1_score
import uuid

In [ ]:
categories = [
    "бытовая техника", "обувь", "одежда", "посуда", "текстиль",
    "товары для детей", "украшения и аксессуары", "электроника", "нет товара"
]
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Initializing the model
model_name = "distilbert-base-multilingual-cased"
classifier = pipeline("zero-shot-classification", model=model_name, tokenizer=model_name, device=0 if torch.cuda.is_available() else -1)

In [ ]:
prompt_template = """
Классифицируй следующий отзыв с маркетплейса по категории товара. Категории: {categories}. Если отзыв связан с недоставкой, отменой заказа или не относится к конкретному товару, выбери категорию "нет товара". Используй контекст и примеры ниже для точной классификации.

### Инструкции:
- Проанализируй текст отзыва, обращая внимание на упоминания типа товара, материала, качества, доставки или размера.
- Если товар явно не упомянут или отзыв о недоставке, выбери "нет товара".
- Верни только название категории.

### Примеры:
1. **Отзыв**: "Заказали 14.10.2017, получили 25.10.2017. На мой размер 42, широкий как мешок. Надо было все таки размер S заказать. По поводу качества хороший пуховик. Мех натуральный, съемный."
   - **Категория**: одежда

2. **Отзыв**: "Товар не получила, деньги не вернули!"
   - **Категория**: нет товара

3. **Отзыв**: "Качество очень даже неплохое, синтетика немного похожа на замшу. Но размер просто капелюшчный, на 44 размер XXL еле подошёл."
   - **Категория**: одежда

4. **Отзыв**: "Очень красивый боди, только нужно полностью его снимать, чтобы сходить в уборную."
   - **Категория**: одежда

5. **Отзыв**: "Идеальные носочки! Беру уже второй раз, очень довольна. Качество хорошее, никаких дырок."
   - **Категория**: одежда

6. **Отзыв**: "Качество отличное, доставка до Башкирии 3 недели"
   - **Категория**: одежда (предполагается, так как контекст типичен для одежды)

7. **Отзыв**: "джинсы пришли с пятнами"
   - **Категория**: одежда

8. **Отзыв**: "Товар не пришёл, деньги не вернули."
   - **Категория**: нет товара

9. **Отзыв**: "Цвет черный а не темно синий, обидно. Шапка хорошая."
   - **Категория**: одежда

10. **Отзыв**: "Очень милые носочки, качество не плохое"
    - **Категория**: одежда

### Отзыв для классификации:
"{review}"

### Категория:
"""

In [ ]:
def classify_review(review):
    prompt = prompt_template.format(categories=", ".join(categories), review=review)
    result = classifier(prompt, candidate_labels=categories, hypothesis_template="Этот отзыв относится к категории {}.")
    return result["labels"][0]

In [ ]:
predictions = []
start_time = time.time()
for review in test_df["text"]:
    pred = classify_review(review)
    predictions.append(pred)
end_time = time.time()

In [ ]:
avg_time = (end_time - start_time) / len(test_df)
print(f"Average inference time per review: {avg_time:.2f} seconds")

# Saving predictions to CSV
output_df = pd.DataFrame({"text": test_df["text"], "category": predictions})
output_df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")

In [ ]:
print("\nSample predictions:")
print(output_df.head())